<h1>位置命令、回授工具箱</h1>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as plty
import plotly.graph_objects as go
import scipy.ndimage as ndimage
import scipy.fft as fft

In [2]:
class Single_Axis:
    def __init__(self, filename, time_interval) -> None:
        self.data = self.read_measure_file(filename)
        self.time_interval = time_interval                    #確認資料點間隔時間
        self.moving_posc = None
        self.moving_posf = None
        self.get_posc_and_posf()
        self.VT_data = None
        self.AT_data = None
        self.JT_data = None
        pass
    
    def read_measure_file(self, filename):
        return pd.read_csv(filename, skiprows=[0, 1, 3])
    
    def get_key_info_from_df(self, key, threshold = 5):       #自動挑選位移軸向
        pos_info = self.data
        column_pos = []
        pos_moving = None
        for ele in pos_info.columns:
            if key in ele:
                column_pos.append(ele)
        if len(column_pos) > 1:
            for pos in column_pos:
                for ele in pos_info[pos]:
                    if ele > threshold or ele < -threshold:
                        pos_moving = pos_info[pos]
                        break
        else:
            pos_moving = pos_info[key]
        return pos_moving

    def get_posc_and_posf(self):                             #自動選擇位置命令、位置回授
        self.moving_posc = self.get_key_info_from_df("POSC")
        self.moving_posf = self.get_key_info_from_df("POSF")

    def derivative(self, data):                              #(x2-x1)/(t2-t1)
        new_data = np.array([0.])
        for i in range(1, len(data)):
            new_data = np.append(new_data, (data[i]-data[i-1])/self.time_interval)
        new_data = ndimage.uniform_filter(new_data, 5)            #計算加速度時要進行濾波
        return new_data
    
    def fanuc_rfft(self, data):                              #發那科-傅立葉轉換
        dB = fft.rfft(data)
        dB = np.abs(fft.rfft(data, norm="forward"))*2  #核心函數，正規化方式另外乘以2即可對應發那科頻響
        freq = fft.rfftfreq(len(data), self.time_interval)
        return dB, freq

f = "./X-axis0430_F8000_P2021_200_P1825_5500_P1660_800_P1772_8.csv"
f2 = "./FFG_F8000_P1660_1200_P1772_32.csv"
test_class = Single_Axis(f, 0.001)
test_class2 = Single_Axis(f2, 0.001)

In [3]:
temp_VT = test_class.derivative(test_class.moving_posc)
temp_AT = test_class.derivative(temp_VT)
# temp_AT = ndimage.uniform_filter(temp_AT, 3)
temp_VT2 = test_class2.derivative(test_class2.moving_posc)
temp_AT2 = test_class2.derivative(temp_VT2)

In [4]:
temp_AT_fft_db, temp_AT_fft_freq = test_class.fanuc_rfft(temp_AT)
plty.line(y=temp_AT_fft_db, x=temp_AT_fft_freq)

In [5]:
temp_AT_fft = fft.rfft(temp_AT)
temp_AT_fft2 = fft.rfft(temp_AT2)
phase = np.angle(temp_AT_fft)/np.pi*180
phase2 = np.angle(temp_AT_fft2)/np.pi*180
fig = go.Figure(data=go.Scatter(y=phase, x=temp_AT_fft_freq, mode='lines+markers', name=f"PMC_Tapping"))
fig.add_trace(go.Scatter(y=phase2, x=temp_AT_fft_freq, mode='lines+markers', line_color="#BB6611", name=f"FFG"))


In [6]:
import cmath
phase = [0.0]
sum = 0
for i in range(len(temp_AT_fft-1)):
    if cmath.phase(temp_AT_fft[i]) > cmath.phase(temp_AT_fft[i-1]):
        sum -= cmath.phase(temp_AT_fft[i])/np.pi*180
    else:
        sum += cmath.phase(temp_AT_fft[i])/np.pi*180
    phase.append(sum)
plty.scatter({"index":[x for x in range(len(temp_AT_fft))], "y":np.imag(temp_AT_fft), "x":np.real(temp_AT_fft)}, x="x", y="y", color="index")

In [7]:
np.angle(temp_AT_fft[5])/np.pi*180+90

173.21364143592444